In [1]:

# all of the imports
import pandas as pd
import numpy as np
import pickle 
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
% matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [10]:
cars=pd.read_pickle('cars2frame.pkl')
cars.corr()


,length,width,height,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,...,make[T.nissan],make[T.peugot],make[T.plymouth],make[T.porsche],make[T.renault],make[T.saab],make[T.subaru],make[T.toyota],make[T.volkswagen],make[T.volvo]
length,1.000000,0.858084,0.496218,0.609437,0.118664,0.160172,0.583813,-0.280986,-0.689660,-0.719324,...,-0.083746,0.331643,-0.145087,-0.062295,NaN,0.176723,-0.111089,-0.082694,-0.035467,0.285739
width,0.858084,1.000000,0.315834,0.544311,0.186432,0.190997,0.616779,-0.251627,-0.647099,-0.692220,...,-0.119532,0.287928,-0.146487,-0.004161,NaN,0.051420,-0.112705,-0.165717,-0.032444,0.238812
height,0.496218,0.315834,1.000000,0.189283,-0.055525,0.261160,-0.084412,-0.264078,-0.102367,-0.151188,...,-0.030441,0.339587,-0.152562,-0.158088,NaN,0.166833,-0.011948,-0.025885,0.141585,0.242889
bore,0.609437,0.544311,0.189283,1.000000,-0.066793,0.003057,0.568527,-0.277662,-0.591950,-0.600040,...,-0.088131,0.228432,-0.147980,0.245818,NaN,0.028877,0.274439,-0.080693,-0.188286,0.300566
stroke,0.118664,0.186432,-0.055525,-0.066793,1.000000,0.199882,0.100040,-0.068300,-0.027641,-0.036453,...,0.064150,-0.070476,0.077236,-0.137555,NaN,-0.197320,-0.517889,0.006636,0.122347,-0.080408
compression-ratio,0.160172,0.190997,0.261160,0.003057,0.199882,1.000000,-0.214401,-0.444582,0.331413,0.267941,...,-0.031525,0.229620,-0.072559,-0.024823,NaN,-0.043681,-0.087112,0.015928,0.216784,0.001949
horsepower,0.583813,0.616779,-0.084412,0.568527,0.100040,-0.214401,1.000000,0.105654,-0.834117,-0.812917,...,-0.006047,-0.022356,-0.084584,0.336106,NaN,0.110354,-0.115397,-0.123056,-0.150424,0.160068
peak-rpm,-0.280986,-0.251627,-0.264078,-0.277662,-0.068300,-0.444582,0.105654,1.000000,-0.069493,-0.016950,...,0.053454,-0.225783,0.106446,0.217044,NaN,0.089206,-0.177902,-0.227779,0.029978,0.100207
city-mpg,-0.689660,-0.647099,-0.102367,-0.591950,-0.027641,0.331413,-0.834117,-0.069493,1.000000,0.972350,...,0.081193,-0.111811,0.083668,-0.178472,NaN,-0.140672,0.038460,0.147507,0.128699,-0.160549
highway-mpg,-0.719324,-0.692220,-0.151188,-0.600040,-0.036453,0.267941,-0.812917,-0.016950,0.972350,1.000000,...,0.098472,-0.150924,0.093533,-0.113469,NaN,-0.091750,-0.003422,0.134335,0.153215,-0.180292


In [14]:

y = cars.log_price
X=cars.drop(['log_price','price','stroke'],1)

model= LinearRegression()
model.fit(X,y)
model.score(X,y)

0.95501453303584205

In [15]:
# using cross validation
X_train, X_test, y_train, y_test = train_test_split(X, y)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.8678587139978583

In [16]:
# Student Exercise : Use cross validation for feature selection on the car data 
# 5-fold cross-validation ( As mentioned above, 10-fold is typically ideal, but this is a relatively small data set)
# Use r2 as the scoring criteria, 
# Note the Improvement guidelines below

In [19]:
from sklearn.cross_validation import cross_val_score

# 10-fold cross-validation with our fake data
reg = LinearRegression()
scores = cross_val_score(reg, X, y, cv=5, scoring='mean_squared_error')

# scores output is negative, a sklearn quirk bc mse is used to min. optimization func.
print(np.mean(-scores))

0.0601511874087
